In [1]:
import numpy as np
import pandas as pd

In [2]:
import pandas as pd
import json

file_path = "/home/ritik/Documents/xploremate/synthetic_data_NLP_queries_webScraping/synthesis.json"

with open(file_path, 'r') as file:
    data = json.load(file)

texts = []
classes = []

for label, texts_list in data.items():
    texts.extend(texts_list) 
    classes.extend([label] * len(texts_list))

df0 = pd.DataFrame({'text': texts, 'class': classes})

print(df0.head())


                                                text     class
0  What are the effects of acquisition on PG's ca...  relevant
1  What are the advantages of using the Price Osc...  relevant
2  Assess the use of the High-Low Index in Amgen ...  relevant
3  What are the psychological factors contributin...  relevant
4  What are the implications of using the Ulcer I...  relevant


In [3]:
df = df0.sample(frac=1, random_state = 4).reset_index(drop=True)

print(df.head())

                                                text       class
0  He says heâs likely to set fire to the barn ...  irrelevant
1  What are the benefits and drawbacks of using t...    relevant
2  Fast alongside a wharf,\r\nlittered like any s...  irrelevant
3  Rank MS, C, and Kimberly-Clark by capital adeq...    relevant
4  Analyze how Mondelez International Inc. uses t...    relevant


In [4]:
df['text'].replace(['.', '', ' '], np.nan, inplace=True)

/tmp/ipykernel_3800/1862367166.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['text'].replace(['.', '', ' '], np.nan, inplace=True)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39070 entries, 0 to 39069
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    39070 non-null  object
 1   class   39070 non-null  object
dtypes: object(2)
memory usage: 610.6+ KB


In [6]:
df['class'].value_counts()

class
irrelevant    20000
relevant      19070
Name: count, dtype: int64

In [7]:
df['class'] = df['class'].map({'relevant': 1, 'irrelevant': 0})


In [8]:
df.head()

,text,class
0,He says heâs likely to set fire to the barn ...,0
1,What are the benefits and drawbacks of using t...,1
2,"Fast alongside a wharf,\r\nlittered like any s...",0
3,"Rank MS, C, and Kimberly-Clark by capital adeq...",1
4,Analyze how Mondelez International Inc. uses t...,1


In [9]:
# !pip install spacy

In [10]:
import pandas as pd
import re
import contractions
from bs4 import BeautifulSoup
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import spacy

In [11]:
# !python -m spacy download en_core_web_sm

In [12]:
nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ritik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ritik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ritik/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/ritik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
stop_words = set(stopwords.words('english'))

In [14]:
print(stop_words)

{"it's", 'had', 'no', 'over', 'just', 'don', 'can', 'our', 'a', 'mustn', "won't", 'your', 'why', 'further', "couldn't", 'it', 'through', "weren't", 'yours', 'yourselves', 'against', 'her', 't', 'doing', 'up', "needn't", 'down', 'about', 've', 'mightn', 'but', 'same', 'off', 'with', 'ourselves', 'shouldn', 'few', 'he', 'wouldn', 'there', 'herself', 'him', 'was', 'hasn', 'some', 'such', 'very', 'then', 'll', 'all', 'is', 'once', 'wasn', "shan't", 'while', 'too', 'o', 'of', 'needn', 'my', "haven't", 're', "isn't", "doesn't", 'not', 'been', 'm', 'did', 'has', 'above', 'in', 'were', "hasn't", "mustn't", 'at', 'again', 'into', "you'd", 'having', 'under', 'yourself', 'these', 'aren', 'doesn', "mightn't", 'own', 'which', 'are', "she's", 'an', 'hadn', 'isn', 'between', "don't", 'each', 'didn', 'ain', 'haven', "aren't", 'than', "wouldn't", 'because', 'his', 'where', "didn't", 'when', "hadn't", 'shan', 'their', 'or', 'other', 'more', 'those', 'she', 'myself', 'here', 'they', 'being', 'what', 'as'

In [15]:
from nltk.corpus import words

nltk.download('words')

eng_words = set(words.words())

def lowercase_text(text):
    def process_word(word):
        if 2 <= len(word) <= 10 and word.isupper() and word.lower() not in eng_words:
            return word 
        else:
            return word.lower()

    words_in_text = text.split()
    processed_words = [process_word(word) for word in words_in_text]
    
    return ' '.join(processed_words)


[nltk_data] Downloading package words to /home/ritik/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [16]:
def remove_punctuation(text, keep_punctuation=None):
    """Remove punctuation from text, but optionally keep some specified punctuation."""
    if keep_punctuation is None:
        keep_punctuation = {'.', ',', ':', '-', "'", '(', ')', '/', '?'}
        
    text = re.sub(r'[\n\t]', ' ', text)
    pattern = r'[^\w\s' + ''.join(re.escape(p) for p in keep_punctuation) + r']'
    text = re.sub(pattern, ' ', text)
    return text

In [17]:
def remove_html_urls(s):
    try:
        if '<' in s and '>' in s:
            soup = BeautifulSoup(s, 'html.parser')
            cleaned_text = soup.get_text(separator=' ')
        else:
            cleaned_text = s

        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        cleaned_text = re.sub(url_pattern, '', cleaned_text)

        return cleaned_text
    except Exception as e:
        print(f"Error in remove_html_urls: {str(e)}")
        return s


In [18]:
def expand_contractions(text):
    try:
        # print("Step 4: Expanding contractions.")
        expanded_text = contractions.fix(text)
        return expanded_text
    except Exception as e:
        print(f"Error in expand_contractions: {str(e)}")
        return text

In [19]:
def remove_extra_whitespace(text):
    try:
        # print("Step 5: Removing extra whitespaces.")
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    except Exception as e:
        print(f"Error in remove_extra_whitespace: {str(e)}")
        return text

In [20]:
stopwords_to_remove = [
    'a', 'an', 'the', 'and', 'but', 'or', 'if', 'because', 'as', 'while', 'of', 'at', 
    'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 
    'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 
    'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 
    'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very'
]

In [21]:
def remove_stopwords(text, custom_stopwords=None):
    """Remove a custom set of stopwords from text, while retaining important auxiliary verbs."""
    if custom_stopwords is None:
        custom_stopwords = stopwords_to_remove
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in custom_stopwords]
    return ' '.join(filtered_words)


In [22]:
def normalize_text(text):
    try:
        # print("Step 7: Normalizing text.")
        normalized_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return normalized_text
    except Exception as e:
        print(f"Error in normalize_text: {str(e)}")
        return text

In [23]:
def lemmatizer_with_pos(s):
    try:
        # print("Step 8: Lemmatizing text with POS tags.")
        words = word_tokenize(s)
        lemmatizer = WordNetLemmatizer()
        
        def get_wordnet_pos(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return None

        tagged_words = pos_tag(words)
        lemmatized_words = []
        
        for word, tag in tagged_words:
            wn_pos = get_wordnet_pos(tag) or wordnet.NOUN
            lemmatized_word = lemmatizer.lemmatize(word, pos=wn_pos)
            lemmatized_words.append(lemmatized_word)
        
        return ' '.join(lemmatized_words)
    except Exception as e:
        print(f"Error in lemmatizer_with_pos: {str(e)}")
        return s

In [24]:
def preprocess_text(text):
    try:
        if not text or text.strip() == '':
            return ''
        
        
        text = expand_contractions(text)
        text = remove_html_urls(text)
        text = remove_punctuation(text)
        text = remove_extra_whitespace(text)
        text = normalize_text(text)
        text = lemmatizer_with_pos(text)
        text = remove_stopwords(text)
        text = lowercase_text(text)
        
        return text
    except Exception as e:
        print(f"Error during preprocessing: {str(e)}")
        return text

In [25]:
example_text = "This is a sample text! We'll be removing AAPL <html>tags</html>, expanding contractions like can't, and lemmatizing words."
preprocessed_text = preprocess_text(example_text)

In [26]:
print("\nPreprocessed Text:\n", preprocessed_text)


Preprocessed Text:
 this be sample text we will be remove AAPL tag , expand contraction like can , lemmatizing word .


In [27]:
from tqdm import tqdm
tqdm.pandas()

def preprocess_dataframe(df):
    tqdm.pandas(desc="Processing Text")
    df['text'] = df['text'].progress_apply(preprocess_text)
    return df

In [28]:
# df = preprocess_dataframe(df)

In [29]:
# df.head()

In [30]:
df['text'][1]

'What are the benefits and drawbacks of using the Percentage Price Oscillator (PPO) for PPL’s trading strategy? Analyze its impact on market trend analysis.'

In [31]:
df0 = preprocess_dataframe(df0)

Processing Text: 100%|███████████████████| 39070/39070 [00:42<00:00, 915.75it/s]


In [35]:
import torch
import logging
from transformers import BertTokenizer, BertModel
from tqdm import tqdm 

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("Loading financial model and tokenizer...")
financial_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
financial_model = BertModel.from_pretrained('yiyanghkust/finbert-tone')

logging.info("Loading general model and tokenizer...")
general_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
general_model = BertModel.from_pretrained('bert-base-uncased')

financial_model.eval()
general_model.eval()

2024-09-21 11:42:11,949 - INFO - Loading financial model and tokenizer...
/home/ritik/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-09-21 11:42:14,050 - INFO - Loading general model and tokenizer...


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [38]:
texts = df['text']

financial_texts = financial_texts.dropna().astype(str).tolist()
non_financial_texts = non_financial_texts.dropna().astype(str).tolist()


def generate_embeddings(texts, tokenizer, model, batch_size=32, max_length=512):
    all_embeddings = []
    num_batches = (len(texts) + batch_size - 1) // batch_size 

    for i in tqdm(range(0, len(texts), batch_size), desc="Processing batches"):
        try:
            batch_texts = texts[i:i + batch_size]
            logging.info(f"Processing batch {i // batch_size + 1} with {len(batch_texts)} texts.")

            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=max_length)

            with torch.no_grad():
                outputs = model(**inputs)

            batch_embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeddings.append(batch_embeddings)

            logging.info(f"Batch {i // batch_size + 1} processed successfully.")
        
        except Exception as e:
            logging.error(f"Error processing batch {i // batch_size + 1}: {e}")

    all_embeddings = torch.cat(all_embeddings, dim=0)
    
    return all_embeddings

In [39]:
for i, text in enumerate(financial_texts):
    if not isinstance(text, str):
        logging.error(f"Invalid data type at index {i}: {type(text)}")


In [40]:
batch_size = 16

try:
    logging.info("Generating financial embeddings...")
    financial_embeddings = generate_embeddings(financial_texts, financial_tokenizer, financial_model, batch_size=batch_size)
    logging.info("Generated financial embeddings successfully.")
    logging.info(f"Generated financial embeddings shape: {financial_embeddings.shape}")

except Exception as e:
    logging.error(f"Error generating financial embeddings: {e}")

try:
    logging.info("Generating non-financial embeddings...")
    non_financial_embeddings = generate_embeddings(non_financial_texts, general_tokenizer, general_model, batch_size=batch_size)
    logging.info("Generated non-financial embeddings successfully.")
    logging.info(f"Generated non-financial embeddings shape: {non_financial_embeddings.shape}")

except Exception as e:
    logging.error(f"Error generating non-financial embeddings: {e}")


2024-09-21 11:44:06,156 - INFO - Generating financial embeddings...
Processing batches:   0%|                              | 0/1192 [00:00<?, ?it/s]2024-09-21 11:44:06,158 - INFO - Processing batch 1 with 16 texts.
2024-09-21 11:44:06,999 - INFO - Batch 1 processed successfully.
Processing batches:   0%|                      | 1/1192 [00:00<16:42,  1.19it/s]2024-09-21 11:44:07,000 - INFO - Processing batch 2 with 16 texts.
2024-09-21 11:44:07,410 - INFO - Batch 2 processed successfully.
Processing batches:   0%|                      | 2/1192 [00:01<11:40,  1.70it/s]2024-09-21 11:44:07,411 - INFO - Processing batch 3 with 16 texts.
2024-09-21 11:44:07,697 - INFO - Batch 3 processed successfully.
Processing batches:   0%|                      | 3/1192 [00:01<08:55,  2.22it/s]2024-09-21 11:44:07,698 - INFO - Processing batch 4 with 16 texts.
2024-09-21 11:44:08,059 - INFO - Batch 4 processed successfully.
Processing batches:   0%|                      | 4/1192 [00:01<08:13,  2.41it/s]2024

In [41]:
financial_embeddings_np = financial_embeddings.cpu().numpy()
non_financial_embeddings_np = non_financial_embeddings.cpu().numpy()

all_embeddings_np = np.concatenate([financial_embeddings_np, non_financial_embeddings_np], axis=0)

assert len(all_embeddings_np) == len(df0), "Mismatch between embeddings and rows in df0"

df0['embeddings'] = list(all_embeddings_np)

In [42]:
df0.to_csv('df0_with_embeddings.csv', index=False)

print("DataFrame saved to 'df0_with_embeddings.csv'")

DataFrame saved to 'df0_with_embeddings.csv'
